# Parcial final: Daniel Meneses Rojas C.C. 1036671105

### Modelo QML Titanic Original

### Ansatz= **StronglyEntanglingLayers**
### Optimizador= **GradientDescentOptimizer**

## Librerias

In [1]:
import math                                                 # Libreria para calculos matemáticos
import pandas as pd                                         # Libreria para manipular de datos
import pennylane as qml                                     # Libreria necesaria para computación cuántica
from pennylane import numpy as np                           # Versión diferenciable de numpy
from pennylane.optimize import GradientDescentOptimizer     # Optimizador básico

# Libreria para dividir los datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split        
# Métricas de evaluación
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


## **I.** Preprocesamiento de los datos de entrada

In [2]:
df_train = pd.read_csv('train.csv')                      # Carga el dataset desde un archivo CSV
df_train['Pclass'] = df_train['Pclass'].astype(str)      # Convierte la columna 'Pclass' a tipo string

# Codificación one-hot para las variables categóricas
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)

# Llena los valores faltantes de 'Age' con la mediana
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

# Crea una nueva columna para identificar si es niño (<12 años)
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)

# Selecciona las variables para el modelo
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

## **II.** División de datos de entrada y datos de prueba

In [3]:
# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])  # División estratificada

# Conversión a arrays de Pennylane (diferenciables)
X_train = np.array(X_train.values, requires_grad=False)

# Conversión de etiquetas {0,1} a {-1,1} (necesario para el clasificador cuántico)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)


## **III.** Etapa del Feature Map

In [4]:
# Configuración del dispositivo cuántico con 4 qubits
num_qubits = 4
num_layers = 4
dev = qml.device("default.qubit", wires=num_qubits)         # Simulador de qubits

# Etapa del Feature Map
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(num_qubits))      # Codifica los datos clásicos como estados base


## **IV.** Etapa del Ansatz

In [5]:
# Definición del circuito cuántico completo
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)                         # Aplica el feature map (codificación)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))                        # Aplica cada capa del ansatz
    return qml.expval(qml.PauliZ(0))            # Retorna la expectativa de PauliZ en el qubit 0

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias           # Predicción = salida del circuito + sesgo

## **V.** Etapa de funcion de Pérdida

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2           # Error cuadrático por muestra
    return loss / len(labels)         # Promedio del error

## **VI.** Etapa de evaluación de la Función de Costo

In [7]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]     # Clasifica todos los datos
    return square_loss(Y, predictions)                                      # Calcula el costo

## **VII.** Etapa de Análisis de Precisión

In [8]:
def accuracy(labels, predictions):
    correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:               # Cuenta como correcta si predicción y etiqueta coinciden
            correct += 1
    return correct / len(labels)            # Porcentaje de aciertos

## **VIII.** Pesos iniciales definidos

In [9]:
# Semilla para reproducibilidad
np.random.seed(0)               

# Inicialización aleatoria de pesos
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)  

# Sesgo inicial
bias_init = np.array(0.0, requires_grad=True)  

## **IX.** Pesos finales entrenados

In [10]:
opt = GradientDescentOptimizer(stepsize=0.1)
num_it = 25
batch_size = math.floor(len(X_train) / num_it)

weights = weights_init
bias = bias_init

for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    
    weights, bias = opt.step(lambda w, b: cost(w, b, X_batch, Y_batch), weights, bias)

    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f}".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )


Iter:     1 | Cost: 1.8240705 | Accuracy: 0.5255930
Iter:     2 | Cost: 1.7694584 | Accuracy: 0.5255930
Iter:     3 | Cost: 1.7476024 | Accuracy: 0.5255930
Iter:     4 | Cost: 1.7372305 | Accuracy: 0.5255930
Iter:     5 | Cost: 1.7315333 | Accuracy: 0.5255930
Iter:     6 | Cost: 1.7193486 | Accuracy: 0.5255930
Iter:     7 | Cost: 1.7160961 | Accuracy: 0.5255930
Iter:     8 | Cost: 1.7108819 | Accuracy: 0.5255930
Iter:     9 | Cost: 1.7070357 | Accuracy: 0.5255930
Iter:    10 | Cost: 1.6918464 | Accuracy: 0.5255930
Iter:    11 | Cost: 1.6772090 | Accuracy: 0.5255930
Iter:    12 | Cost: 1.6093775 | Accuracy: 0.5255930
Iter:    13 | Cost: 1.5710094 | Accuracy: 0.5255930
Iter:    14 | Cost: 1.4893539 | Accuracy: 0.5255930
Iter:    15 | Cost: 1.4072849 | Accuracy: 0.5255930
Iter:    16 | Cost: 1.3249171 | Accuracy: 0.5255930
Iter:    17 | Cost: 1.1882233 | Accuracy: 0.5255930
Iter:    18 | Cost: 1.1125386 | Accuracy: 0.5255930
Iter:    19 | Cost: 1.0375188 | Accuracy: 0.7240949
Iter:    20 

## **X.** Evaluación de desempeño del clasificador cuantico final

In [11]:
# Evaluación de desempeño del clasificador
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# Realiza las predicciones con el modelo de entrenado
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

# Se evalua el desempeño con las metricas de exactitud, precisión y recall
print("Accuracy Score:", accuracy_score(Y_test, predictions))
print("Precision Score:", precision_score(Y_test, predictions))
print("Recall Score:", recall_score(Y_test, predictions))
print("F1 Score (macro):", f1_score(Y_test, predictions, average='macro'))


Accuracy Score: 0.6777777777777778
Precision Score: 1.0
Recall Score: 0.17142857142857143
F1 Score (macro): 0.5420249166520442


## Comparación de los dos modelos

### Tabla de comparacion de los dos modelos

| **Métrica**   | **Modelo Original** | **Modelo Modificado** | **Observación** |
| ------------- | ------------------- | --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **Accuracy**  | 78.89%              | 67.80%                | El modelo modificado clasifica correctamente una menor cantidad de ejemplos totales.                                                      |
| **Precision** | 76.77%              | 100%                  | El modelo modificado nunca comete falsos positivos, pero esto puede deberse a que predice muy pocos positivos.                              |
| **Recall**    | 65.71%              | 17.10%                | El modelo modificado identifica solo una pequeña parte de los verdaderos positivos, lo que sugiere una alta tasa de falsos negativos.  |
| **F1 Score**  | 77.12%              | 54.20%                | El modelo modificado tiene un rendimiento general más bajo, debido al desbalance entre precisión y recall.                                     |

